# This is an Notebook which can evalute multiple files.  It implements a number of features:

- TopK
- Removing recent referrals
- Thresholding 
- Tables and visualization
- Config files for standardized processes. 



In [83]:
%reload_ext autoreload
%autoreload 2
#ignore warnings.
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

#imports
import pandas as pd
import os, sys
import importlib
import yaml, json
from pathlib import Path

#path append
sys.path.append(os.path.join(Path.cwd(), 'modules'))
import Evaluate, Helper, Present, Score, Synthetic


<H3> Imports</H3>

### File configuration
<p> Just edit the configuration file path/address. prediction_files contain configs for different prediction files. config_file contains other configs for referral, visualization, experiment etc. </p>

In [84]:
config_file = 'config/config.yaml'
prediction_config_files = ['config/predictions/pred1.yaml', 'config/predictions/pred2.yaml']
generate_data=False


### Load Configuration and Referrals

In [85]:
#Load Configuration
c_r, c_e, c_gen, c_aws, c_visual, predictions=Helper.load_configuration(config_file, prediction_config_files)

#Generate Data (if required)
if generate_data:
    Synthetic.generate_synthetic_event_data(c_gen) 
    Synthetic.generate_synthetic_prediction_data(c_gen)
    
#Load Referral Data
referrals=Helper.read_file(directory=c_r['dir'],file=c_r['file'],file_format=c_r['file_format'],aws=c_r['aws'],bucket= c_r['bucket'])


### Evaluate Models
For each configuration file provided it will evaluate the models. 

In [86]:
for c_p in predictions:    
    prediction=Helper.read_file(directory=c_p['dir'],file=c_p['file'],file_format=c_p['file_format'],aws=c_p['aws'],bucket= c_p['bucket'])
    
    #TBD Implement a check that there is a match between prediction and evaluation file.
    
    all_model_evaluations = Evaluate.evaluate(c_p, c_e, c_r, referrals, prediction) 
    Present.present_evaluation(c_p, c_r, c_e, c_visual, all_model_evaluations)

,Experiment Name,Model,Window,Eval Date,Num Samples,TN_@k=50,FP_@k=50,FN_@k=50,TP_@k=50,TN_@k=60,FP_@k=60,FN_@k=60,TP_@k=60,TN_@p>=0.5,FP_@p>=0.5,FN_@p>=0.5,TP_@p>=0.5,TN_@p>=0.6,FP_@p>=0.6,FN_@p>=0.6,TP_@p>=0.6,precision_@k=50,precision_@k=60,precision_@p>=0.5,precision_@p>=0.6,recall_@k=50,recall_@k=60,recall_@p>=0.5,recall_@p>=0.6,accuracy_@k=50,accuracy_@k=60,accuracy_@p>=0.5,accuracy_@p>=0.6,balanced_acc_@k=50,balanced_acc_@k=60,balanced_acc_@p>=0.5,balanced_acc_@p>=0.6,f1_score_@k=50,f1_score_@k=60,f1_score_@p>=0.5,f1_score_@p>=0.6,log_loss,roc_auc_score,brier_score_loss,Prediction Source,Referral Source,Result Output,Start Time,End Time,Total Time
0,Version 1.0,lin_reg,2017/01-2017/04,2017-01-01,495,442.00000,49.00000,3.00000,1.00000,433.00000,58.00000,2.00000,2.00000,236.00000,255.00000,2.00000,2.00000,299.00000,192.00000,2.00000,2.00000,0.02000,0.03333,0.00778,0.01031,0.25000,0.50000,0.50000,0.50000,0.25000,0.50000,0.50000,0.50000,0.57510,0.69094,0.49033,0.55448,0.03704,0.06250,0.01533,0.02020,0.98694,0.52749,0.32991,./data/simulated_prediction_data/predictions.csv,./data/simulated_prediction_data/referrals.csv,./results/results.csv,2020-06-03 02:17:23.033965-04:00,2020-06-03 02:17:23.082834-04:00,00:00:00.048869
1,Version 1.0,lin_reg,2017/01-2017/07,2017-01-01,495,442.00000,49.00000,3.00000,1.00000,433.00000,58.00000,2.00000,2.00000,236.00000,255.00000,2.00000,2.00000,299.00000,192.00000,2.00000,2.00000,0.02000,0.03333,0.00778,0.01031,0.25000,0.50000,0.50000,0.50000,0.25000,0.50000,0.50000,0.50000,0.57510,0.69094,0.49033,0.55448,0.03704,0.06250,0.01533,0.02020,0.98694,0.52749,0.32991,./data/simulated_prediction_data/predictions.csv,./data/simulated_prediction_data/referrals.csv,./results/results.csv,2020-06-03 02:17:23.086844-04:00,2020-06-03 02:17:23.102781-04:00,00:00:00.015937
2,Version 1.0,lin_reg,2017/01-2018/01,2017-01-01,495,439.00000,49.00000,6.00000,1.00000,430.00000,58.00000,5.00000,2.00000,235.00000,253.00000,3.00000,4.00000,298.00000,190.00000,3.00000,4.00000,0.02000,0.03333,0.01556,0.02062,0.14286,0.28571,0.57143,0.57143,0.14286,0.28571,0.57143,0.57143,0.52122,0.58343,0.52649,0.59104,0.03509,0.05970,0.03030,0.03980,0.98211,0.59456,0.32780,./data/simulated_prediction_data/predictions.csv,./data/simulated_prediction_data/referrals.csv,./results/results.csv,2020-06-03 02:17:23.103802-04:00,2020-06-03 02:17:23.118747-04:00,00:00:00.014945
3,Version 1.0,rand_forest,2017/01-2017/04,2017-01-01,495,443.00000,48.00000,2.00000,2.00000,433.00000,58.00000,2.00000,2.00000,250.00000,241.00000,2.00000,2.00000,304.00000,187.00000,2.00000,2.00000,0.04000,0.03333,0.00823,0.01058,0.50000,0.50000,0.50000,0.50000,0.50000,0.50000,0.50000,0.50000,0.70112,0.69094,0.50458,0.55957,0.07407,0.06250,0.01619,0.02073,0.98813,0.64358,0.32953,./data/simulated_prediction_data/predictions.csv,./data/simulated_prediction_data/referrals.csv,./results/results.csv,2020-06-03 02:17:23.119748-04:00,2020-06-03 02:17:23.133704-04:00,00:00:00.013956
4,Version 1.0,rand_forest,2017/01-2017/07,2017-01-01,495,443.00000,48.00000,2.00000,2.00000,433.00000,58.00000,2.00000,2.00000,250.00000,241.00000,2.00000,2.00000,304.00000,187.00000,2.00000,2.00000,0.04000,0.03333,0.00823,0.01058,0.50000,0.50000,0.50000,0.50000,0.50000,0.50000,0.50000,0.50000,0.70112,0.69094,0.50458,0.55957,0.07407,0.06250,0.01619,0.02073,0.98813,0.64358,0.32953,./data/simulated_prediction_data/predictions.csv,./data/simulated_prediction_data/referrals.csv,./results/results.csv,2020-06-03 02:17:23.135703-04:00,2020-06-03 02:17:23.149666-04:00,00:00:00.013963
5,Version 1.0,rand_forest,2017/01-2018/01,2017-01-01,495,440.00000,48.00000,5.00000,2.00000,430.00000,58.00000,5.00000,2.00000,249.00000,239.00000,3.00000,4.00000,302.00000,186.00000,4.00000,3.00000,0.04000,0.03333,0.01646,0.01587,0.28571,0.28571,0.57143,0.42857,0.28571,0.28571,0.57143,0.42857,0.59368,0.58343,0.54084,0.52371,0.07018,0.05970,0.03200,0.03061,0.98659,0.61300,0.32879,./data/simulated_prediction_data/predictions.csv,./

,Experiment Name,Model,Window,Eval Date,Num Samples,TN_@k=50,FP_@k=50,FN_@k=50,TP_@k=50,TN_@k=60,FP_@k=60,FN_@k=60,TP_@k=60,TN_@p>=0.5,FP_@p>=0.5,FN_@p>=0.5,TP_@p>=0.5,TN_@p>=0.6,FP_@p>=0.6,FN_@p>=0.6,TP_@p>=0.6,precision_@k=50,precision_@k=60,precision_@p>=0.5,precision_@p>=0.6,recall_@k=50,recall_@k=60,recall_@p>=0.5,recall_@p>=0.6,accuracy_@k=50,accuracy_@k=60,accuracy_@p>=0.5,accuracy_@p>=0.6,balanced_acc_@k=50,balanced_acc_@k=60,balanced_acc_@p>=0.5,balanced_acc_@p>=0.6,f1_score_@k=50,f1_score_@k=60,f1_score_@p>=0.5,f1_score_@p>=0.6,log_loss,roc_auc_score,brier_score_loss,Prediction Source,Referral Source,Result Output,Start Time,End Time,Total Time
0,Version 1.0,lin_reg,2017/01-2017/04,2017-01-01,495,442.00000,49.00000,3.00000,1.00000,433.00000,58.00000,2.00000,2.00000,236.00000,255.00000,2.00000,2.00000,299.00000,192.00000,2.00000,2.00000,0.02000,0.03333,0.00778,0.01031,0.25000,0.50000,0.50000,0.50000,0.25000,0.50000,0.50000,0.50000,0.57510,0.69094,0.49033,0.55448,0.03704,0.06250,0.01533,0.02020,0.98694,0.52749,0.32991,./data/simulated_prediction_data/predictions2.csv,./data/simulated_prediction_data/referrals.csv,./results/results.csv,2020-06-03 02:17:23.696768-04:00,2020-06-03 02:17:23.741648-04:00,00:00:00.044880
1,Version 1.0,lin_reg,2017/01-2017/07,2017-01-01,495,442.00000,49.00000,3.00000,1.00000,433.00000,58.00000,2.00000,2.00000,236.00000,255.00000,2.00000,2.00000,299.00000,192.00000,2.00000,2.00000,0.02000,0.03333,0.00778,0.01031,0.25000,0.50000,0.50000,0.50000,0.25000,0.50000,0.50000,0.50000,0.57510,0.69094,0.49033,0.55448,0.03704,0.06250,0.01533,0.02020,0.98694,0.52749,0.32991,./data/simulated_prediction_data/predictions2.csv,./data/simulated_prediction_data/referrals.csv,./results/results.csv,2020-06-03 02:17:23.743648-04:00,2020-06-03 02:17:23.759613-04:00,00:00:00.015965
2,Version 1.0,lin_reg,2017/01-2018/01,2017-01-01,495,439.00000,49.00000,6.00000,1.00000,430.00000,58.00000,5.00000,2.00000,235.00000,253.00000,3.00000,4.00000,298.00000,190.00000,3.00000,4.00000,0.02000,0.03333,0.01556,0.02062,0.14286,0.28571,0.57143,0.57143,0.14286,0.28571,0.57143,0.57143,0.52122,0.58343,0.52649,0.59104,0.03509,0.05970,0.03030,0.03980,0.98211,0.59456,0.32780,./data/simulated_prediction_data/predictions2.csv,./data/simulated_prediction_data/referrals.csv,./results/results.csv,2020-06-03 02:17:23.759613-04:00,2020-06-03 02:17:23.774560-04:00,00:00:00.014947
3,Version 1.0,rand_forest,2017/01-2017/04,2017-01-01,495,443.00000,48.00000,2.00000,2.00000,433.00000,58.00000,2.00000,2.00000,250.00000,241.00000,2.00000,2.00000,304.00000,187.00000,2.00000,2.00000,0.04000,0.03333,0.00823,0.01058,0.50000,0.50000,0.50000,0.50000,0.50000,0.50000,0.50000,0.50000,0.70112,0.69094,0.50458,0.55957,0.07407,0.06250,0.01619,0.02073,0.98813,0.64358,0.32953,./data/simulated_prediction_data/predictions2.csv,./data/simulated_prediction_data/referrals.csv,./results/results.csv,2020-06-03 02:17:23.775557-04:00,2020-06-03 02:17:23.790525-04:00,00:00:00.014968
4,Version 1.0,rand_forest,2017/01-2017/07,2017-01-01,495,443.00000,48.00000,2.00000,2.00000,433.00000,58.00000,2.00000,2.00000,250.00000,241.00000,2.00000,2.00000,304.00000,187.00000,2.00000,2.00000,0.04000,0.03333,0.00823,0.01058,0.50000,0.50000,0.50000,0.50000,0.50000,0.50000,0.50000,0.50000,0.70112,0.69094,0.50458,0.55957,0.07407,0.06250,0.01619,0.02073,0.98813,0.64358,0.32953,./data/simulated_prediction_data/predictions2.csv,./data/simulated_prediction_data/referrals.csv,./results/results.csv,2020-06-03 02:17:23.790525-04:00,2020-06-03 02:17:23.806475-04:00,00:00:00.015950
5,Version 1.0,rand_forest,2017/01-2018/01,2017-01-01,495,440.00000,48.00000,5.00000,2.00000,430.00000,58.00000,5.00000,2.00000,249.00000,239.00000,3.00000,4.00000,302.00000,186.00000,4.00000,3.00000,0.04000,0.03333,0.01646,0.01587,0.28571,0.28571,0.57143,0.42857,0.28571,0.28571,0.57143,0.42857,0.59368,0.58343,0.54084,0.52371,0.07018,0.05970,0.03200,0.03061,0.98659,0.61300,0.32879,./data/simulated_prediction_data/predictions2.